In [1]:
import itertools, re
import PySpice.Logging.Logging as Logging
from PySpice.Spice.Netlist import Circuit
from PySpice.Unit import *

In [2]:
Logging.setup_logging()

<Logger PySpice (INFO)>

In [3]:
def empty_ir(title="Candidate"):
    return {"title": title, "nodes": ["in","out","0"], "components": []}

In [4]:
def add_vin_and_load(ir, ac=1.0, Rload=1e2):
    # AC=1 V small-signal source recorded in IR
    ir["components"].append({"kind":"VAC","name":"in", "pos":"in","neg":"0","ac": ac})
    # Tiny "wire" placeholder for in->out path (a rewire anchor for series insertions)
    ir["components"].append({"kind":"R","name":"Rpath","n1":"in","n2":"out","value":1e-3})
    # Output load
    ir["components"].append({"kind":"R","name":"Rload","n1":"out","n2":"0","value": Rload})

In [5]:
from PySpice.Spice.Netlist import SubCircuitFactory

class RC_LowPass(SubCircuitFactory):
    NAME  = 'rc_lp'
    NODES = ('in', 'out', '0')
    def __init__(self, R=1@u_kΩ, C=1@u_uF):
        super().__init__()
        self.R(1, 'in', 'out', R)
        self.C(1, 'out', '0',  C)

In [6]:
SUBCKT_REGISTRY = {
    'rc_lp':  RC_LowPass
}

In [7]:
def emit_pyspice(ir: dict, subckt_registry) -> Circuit:
    
    c = Circuit(ir["title"])

    for comp in ir["components"]:
        k = comp["kind"].upper()
        if k == "R":
            n1 = comp["n1"]; n2 = comp["n2"]
            c.R(comp["name"], n1, n2, comp["value"])

        elif k == "VAC":   # AC source for .ac analysis
            ac_amp = float(comp["ac"])
            #src = c.V(comp["name"], comp["pos"], comp["neg"], amplitude=ac_amp @ u_V)
            c.V(comp["name"], comp["pos"], comp["neg"], f"dc 0 ac {ac_amp}")
            #src.ac_magnitude = ac_amp @ u_V  
            
        elif k == "SUBCKT":
            subckt_class = subckt_registry[comp["model"]]
            sc = subckt_class()
            c.subcircuit(sc)
            c.X(comp["instance"], comp["model"], comp["n1"], comp["n2"], 0)
    return c

In [8]:
def add_subckt_series(ir, model, instance):
    next_idx = sum(1 for c in ir["components"] if c["kind"] == "SUBCKT")
    new_node = f"n{next_idx}"

    for comp in ir["components"]:
        if comp.get("n2") == "out":
            comp["n2"] = new_node
            break

    ir["components"].append({
        "kind": "SUBCKT",
        "instance" : instance,
        "model" : model,
        "n1" : new_node,
        "n2" : "out"
        })

In [9]:
def add_subckt_shunt(ir, model, instance, node="out"):
    ir["components"].append({
        "kind": "SUBCKT",
        "instance": instance,
        "model": model,
        "n1": node,
        "n2": 0
    })

In [10]:
OPS = ["series", "shunt"]
MODELS = {"series":["rc_lp"], "shunt":["rc_lp"]}

In [11]:
def build_candidate(seq_ops, seq_models):
    ir = empty_ir("Candidate")
    add_vin_and_load(ir, ac=1.0, Rload=1e3)
    for step, (op, model) in enumerate(zip(seq_ops, seq_models), start=1):
        if op == "series":
            add_subckt_series(ir, model, f"{step}")
        else:
            add_subckt_shunt(ir, model, f"{step}", node="out")
    return ir

In [12]:
def enumerate_all(max_steps=3):
    all_circuits = []
    for i in range(1, max_steps+1):
        for ops in itertools.product(OPS, repeat=i):
            choice_lists = [MODELS[o] for o in ops]
            for models in itertools.product(*choice_lists):
                ir = build_candidate(ops, models)
                ckt = emit_pyspice(ir, SUBCKT_REGISTRY)
                all_circuits.append({"ops":ops, "models":models, "ir":ir, "circuit":ckt})
    return all_circuits

    

In [13]:
def save_all_to_one(cands, filename="all_candidates.txt"):
    with open(filename, "w", encoding="utf-8") as f:
        f.write(f"Total candidates: {len(cands)}\n\n")
        for i, c in enumerate(cands, 1):
            ops_str   = "->".join(c["ops"])
            models_str = " ".join(c["models"])
            f.write(f"=== Candidate {i} ===\n")
            f.write(f"ops:   {ops_str}\n")
            f.write(f"kinds: {models_str}\n")


            f.write("\n")
    print(f"All candidates written to {filename}")

In [14]:
if __name__ == "__main__":
    cands = enumerate_all(max_steps=3)         # generate everything (no simulation)
    save_all_to_one(cands, "all_candidates.txt")

All candidates written to all_candidates.txt


In [15]:
cands = enumerate_all(max_steps=3)

with open("all_candidates.spice", "w") as f:
    for cand in cands:
        f.write("* --- Candidate ---\n")
        f.write(str(cand["circuit"]))
        f.write("\n\n")